## 9.2 微调
### 9.2.1 热狗识别

In [1]:
%matplotlib inline
import d2lzh as d2l
import numpy as np
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

#### 9.2.1.1 获取数据集

In [2]:
data_dir = '../data'
base_url = 'https://apache-mxnet.s3-accelerate.amazonaws.com/'
fname = keras.utils.get_file(
    fname='hotdog.zip',
    cache_dir=data_dir,
    origin=base_url + 'gluon/dataset/hotdog.zip',
    extract=True)

In [3]:
train_augs = ImageDataGenerator(
    zoom_range=(0.9, 1.1),
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    featurewise_center=True,
    samplewise_center=True,
    featurewise_std_normalization=True,
    samplewise_std_normalization=True,
)

test_augs = ImageDataGenerator(
    featurewise_center=True,
    samplewise_center=True,
    featurewise_std_normalization=True,
    samplewise_std_normalization=True,
)

train_imgs = train_augs.flow_from_directory(
    os.path.join(data_dir, 'datasets/hotdog/train'), 
    target_size=(224, 224), batch_size=32)

test_imgs = test_augs.flow_from_directory(
    os.path.join(data_dir, 'datasets/hotdog/test'), 
    target_size=(224, 224), batch_size=32)

In [4]:
batch_x, batch_y = train_imgs.next()
print(batch_x[0])

#### 9.2.1.2 定义和初始化模型

In [5]:
resnet50 = keras.applications.ResNet50(
    include_top=False, 
    weights='imagenet', 
    classes=2, 
    input_shape=(224, 224, 3), 
    pooling='avg')
x = keras.layers.Dense(2, activation='softmax')(resnet50.output)
model = keras.Model(resnet50.input, x)
model.compile(optimizer='adam', loss=keras.losses.categorical_crossentropy, metrics=['acc'])
model.summary()

In [ ]:
model.fit_generator(train_imgs, validation_data=test_imgs, epochs=5)